# TF-IDF 적용

## 1. konlp의 트위터 형태소 분석기를 이용해서 1차적으로 stopword 제거

In [36]:
keyword1='킹크랩 섹션 리뷰'
keyword2='킹크랩 섹션'
keywords=[keyword1,keyword2]

- custormised konlpy의 경우 [ ( '단어' , '품사' ) , ...... ] 형태로 나오므로 ['단어', .......] 형태로 만들어준다.

In [15]:
from ckonlpy.tag import Twitter
from ckonlpy.tag import Postprocessor

twitter = Twitter()
okt_nouns = Postprocessor(twitter, passtags={'Noun'})

C:\Users\user\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [16]:
okt_result11 = okt_nouns.pos(text1)
okt_result21 = okt_nouns.pos(text2)

In [20]:
okt_result12=[]
for noun in okt_result11:
    real_noun=noun[0]
    okt_result12.append(real_noun)
    
okt_result22=[]
for noun in okt_result21:
    real_noun=noun[0]
    okt_result22.append(real_noun)

In [26]:
text11=' '.join(okt_result12)
text21=' '.join(okt_result22)

##  2. stopword list를 이용해서 2차적으로 stopword 제거

In [27]:
stopwords=pd.read_csv("한글불용어.csv",encoding='utf8')
stopword=stopwords["불용어"].tolist()

In [28]:
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = stopword
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [29]:
text11_nostop=remove_stopwords(text11)
text22_nostop=remove_stopwords(text22)

## 3. TF-IDF 적용

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

corpus=[text11_nostop,text22_nostop]

In [35]:
# ============================================
# -- Get TFIDF
# ============================================
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(corpus)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

data = [];
for i, sent in enumerate(corpus):
    print('====== document[%d] ======' % i)
    data.append([ (token, sp_matrix[i, word2id[token]]) for token in sent.split()])
    print(data[i])

====== document[0] ======
[('헌병', 0.02159895608563762), ('전투', 0.015367835146552333), ('병', 0.046103505439656994), ('동정', 0.02159895608563762), ('유일하', 0.02159895608563762), ('군', 0.046103505439656994), ('단위', 0.04319791217127524), ('소재지', 0.02159895608563762), ('위치', 0.09220701087931399), ('미군', 0.25918747302765144), ('부대', 0.10799478042818811), ('뭐', 0.046103505439656994), ('법', 0.046103505439656994), ('복무', 0.02159895608563762), ('필자', 0.02159895608563762), ('소개', 0.015367835146552333), ('본인', 0.19978185690518033), ('현재', 0.015367835146552333), ('경험', 0.015367835146552333), ('문답', 0.02159895608563762), ('식', 0.046103505439656994), ('디팩밥', 0.02159895608563762), ('디팩', 0.25918747302765144), ('건물', 0.030735670293104665), ('지은', 0.06479686825691286), ('아주', 0.046103505439656994), ('아름', 0.04319791217127524), ('답', 0.046103505439656994), ('샤방샤방하', 0.02159895608563762), ('짬', 0.046103505439656994), ('밥집', 0.02159895608563762), ('니야', 0.02159895608563762), ('렇', 0.046103505439656994), ('디팩

[('거성', 0.025400735161935088), ('수산', 0.09434558774433033), ('판매', 0.11974632290626543), ('킹크랩', 0.4698936390525367), ('섹션', 0.14974632453322598), ('러시아', 0.07983088193751028), ('캄차카', 0.0036286764517050123), ('해역', 0.01451470580682005), ('포획', 0.018143382258525062), ('하자', 0.007745499544822033), ('마자', 0.010886029355115037), ('배', 0.0025818331816073446), ('킹크랩', 0.4698936390525367), ('손질', 0.06168749967898521), ('자숙', 0.07257352903410025), ('배', 0.0025818331816073446), ('급속', 0.018143382258525062), ('선동', 0.010886029355115037), ('제품', 0.15966176387502057), ('율과', 0.01451470580682005), ('선도', 0.03265808806534511), ('발생', 0.018143382258525062), ('여유', 0.010886029355115037), ('제품', 0.15966176387502057), ('국내', 0.010886029355115037), ('반입', 0.007257352903410025), ('제품', 0.15966176387502057), ('킹크랩', 0.4698936390525367), ('부위', 0.043544117420460146), ('부위', 0.043544117420460146), ('제거', 0.03265808806534511), ('무게', 0.01451470580682005), ('이기', 0.01807283227125141), ('킹크랩', 0.46989363905253

In [39]:
for i in range(0, len(data)):
    toCSV=pd.DataFrame(data[i])
    toCSV=toCSV.sort_values(by=1,ascending=False)
    toCSV=toCSV.drop_duplicates()
    real_keyword=keywords[i]
    toCSV.to_csv(real_keyword+" TF_IDF.csv",encoding='ms949')